# Corner detection and 3D calibration
Based on the tutorial [here](https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/sandbox/ludovic/aruco_calibration_rotation.html)

This is an improvement on the previous notebook. In this notebook, the stereo images are processed as pairs.   This is important because during the corner mapping, only the corners that appear in both images must be saved.  

In [ ]:
computer_name = 'gerrie'
#computer_name = 'marcvanzyl'

In [ ]:
import numpy as np
import PIL, os
import cv2 as cv
from cv2 import aruco
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
%matplotlib inline

# The board
The following line of the board was updated to reflect the correct scale of the board.  This is necessary becuase the `board` object is used later   
`board = aruco.CharucoBoard_create(7, 5, .04026, .8*.04026, aruco_dict)`

Notice the board object returned.  Is contains all the magic of the CharUco pattern. For the image interpretation and to calibrate the cameras the cameras need a picture of the board and also information about the picture (ie. the details of the board).  This is all contained in the board object. 

In [ ]:
board_size = '12x8'
if board_size == '7x5':
    chessboard_num_squares_across = 7
    chessboard_num_squares_up = 5
    chessboard_square_size = 0.04026
    chessboard_aruco_ratio = 0.8   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)


elif board_size == '12x8':
    chessboard_num_squares_across = 12
    chessboard_num_squares_up = 8
    chessboard_square_size = 1
    chessboard_aruco_ratio = 0.7   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)



In [ ]:
workdir = "/Users/{}/Downloads/".format(computer_name)

board = aruco.CharucoBoard_create(chessboard_num_squares_across, 
                                  chessboard_num_squares_up, 
                                  chessboard_square_size, 
                                  chessboard_aruco_ratio*chessboard_square_size, 
                                  aruco_dict)

imboard = board.draw((12000, 8000))
cv.imwrite(workdir + "chessboard.tiff", imboard)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.imshow(imboard, cmap = mpl.cm.gray, interpolation = "nearest")
ax.axis("off")
plt.show()

The board object contains all the vectors (pointing from the bottom left corner) to each of the corners present on the board.  The two kinds of objects are **aruco markers** and the **checkerboard**.  The aruco markers are called '`markers`' in the code and documentation.  The term `marker corners` means the set of 4 corners around each aruco maker. 

The aruco markers can be extracted from the board using the `aruco.getBoardObjectAndImagePoints(board, makerCorners command)`. 

Once the algorithm detected the markerCorners then it can interpolate between the marker corners to find the checkerboard corners.  The positions of checkerboard "inside" corners can be extracted using the folowing. 

The corners are labeled starting from 0 (bottom left) and going right 

# After taking many pictures of the board


In [ ]:
datadir = "/Users/{}/Google Drive/ScienceFair2021/Calibration/OFJet/".format(computer_name)

images = np.array([datadir + f for f in os.listdir(datadir) if f.endswith(".jpeg") ])

image_sort_list = []

for image in images:
    image_sort_list.append(int(image.split('OF-')[1].split('.')[0]))
    
images = images[np.argsort(image_sort_list)]

In [ ]:
images

In [ ]:
imR = cv.imread(datadir+'OF-1.jpeg')

fig, axs = plt.subplots(1,1, figsize=(10,18))

axs.imshow(imR)
axs.set_title("RGB".format(chan))

fig, axs = plt.subplots(3,1, figsize=(10,20))

for chan in range(3):
    

    axs[chan].imshow(imR[:,:,chan],cmap='gray')
    
    axs[chan].set_title("color{}".format(chan))

In [ ]:
imR.shape

# Test the files and data

# Find corners on one image
This is just a test using Harris corner dectector to learn how functions work

In [ ]:
filename = datadir+'OF-1.jpeg'

img = cv.imread(filename)
gray_int = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray = np.float32(gray_int)

dst = cv.cornerHarris(gray,2,3,0.05)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
dst = cv.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[255,0,0]

fig, axs = plt.subplots(3, 1, figsize=(18,22))
axs[0].imshow(dst)
axs[0].set_title("Plot of the Harris corner detector matrix")
axs[1].imshow(dst>0.01*dst.max())
axs[1].set_title("Plot of all likely corners dst>0.01*dst.max()")
axs[2].imshow(img)
axs[2].set_title("img with the detected corners in red")



#ax.axis('off')
plt.show()

corners, ids, rejectedImgPoints = cv.aruco.detectMarkers(gray_int, aruco_dict)

One issue with the images was that the contrast was not very high.  See above.  To improve the contrast I found this code online.  clipLimit is a variable that increases the contras

In [ ]:
# adapted from here https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
def increase_contrast(img, clipLimit=3.0, verbose=False):

    if clipLimit>0.0:
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv.cvtColor(img, cv.COLOR_BGR2LAB)
        if verbose:
            cv.imshow("lab",lab)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv.split(lab)
        if verbose:
            cv.imshow('l_channel', l)
            cv.imshow('a_channel', a)
            cv.imshow('b_channel', b)

        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv.createCLAHE(clipLimit=clipLimit, tileGridSize=(8,8))
        cl = clahe.apply(l)
        if verbose:
            cv.imshow('CLAHE output', cl)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv.merge((cl,a,b))
        if verbose:
            cv.imshow('limg', limg)

        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv.cvtColor(limg, cv.COLOR_LAB2BGR)
        if verbose:
            cv.imshow('final', final)
    else:
        final = img.copy()

    return final

In [ ]:
filename = datadir+'OF-1.jpeg'

img = increase_contrast(cv.imread(filename), clipLimit=3.0, verbose=False)
gray_int = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray = np.float32(gray_int)

dst = cv.cornerHarris(gray,2,3,0.05)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
dst = cv.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img_annotated = img.copy()

img_annotated[dst>0.01*dst.max()]=[255,0,0]

fig, axs = plt.subplots(3, 1, figsize=(18,22))
axs[0].imshow(dst)
axs[0].set_title("Plot of the Harris corner detector matrix")
axs[1].imshow(dst>0.01*dst.max())
axs[1].set_title("Plot of all likely corners dst>0.01*dst.max()")
axs[2].imshow(img_annotated)
axs[2].set_title("img with the detected corners in red")



#ax.axis('off')
plt.show()

corners, ids, rejectedImgPoints = cv.aruco.detectMarkers(gray_int, aruco_dict)

### Now using Charuco

This function:
1. finds the locations of the corners of the aruco squares (`cv2.aruco.detectMarkers`)
1. if markers were found interpolates to find the checkerboard markers between them (`cv2.aruco.interpolateCornersCharuco`)
1. zooms into each checkerboard corner to get sub-pixel accuracy using (`cv2.cornerSubPix`)

In [ ]:
def find_checkerboard_corners(img, board, clipLimit=3.0, verbose=False):

    # These are parameters used by the cv2.cornerSubPix function
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

    # increase the contraxt
    img_h_contrast = increase_contrast(img, clipLimit=clipLimit)
 
    # convert the image to grayscale
    gray = cv.cvtColor(img_h_contrast, cv.COLOR_BGR2GRAY)
    fig, axs = plt.subplots(1, 1, figsize=(18,22))
    axs.imshow(gray, cmap='gray')
    
    #detect_params = aruco.CORNER_REFINE_SUBPIX
    detect_params = None


    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv.aruco.detectMarkers(gray, aruco_dict, detect_params)
    
    if verbose:
        print('Found {} aruco marker corners'.format(len(ids)))
    
    if len(ids)>0:
        (retval, charucoCorners,
         charucoIds) = cv.aruco.interpolateCornersCharuco(corners, ids, gray, board, )
        if verbose:
            print('Found {} checker corners'.format(len(charucoIds)))
        if len(charucoIds)>0:
            # SUB PIXEL DETECTION
            for corner in charucoCorners:
                if verbose:
                    print('Sub pixel optimization:')
                    print(corner)
                cv.cornerSubPix(gray, corner,
                                 winSize = (5,5),
                                 zeroZone = (-1,-1),
                                 criteria = criteria)
                if verbose:
                    print(corner)
                    print('+++')
        
    return charucoCorners, charucoIds, gray.shape

        

One big problem is that you need to give the  calibration only coners that appear in both cameras.  Fortunately, the detection returns the ids of the corners in the `charucoIds`. These `charucoIds` correspond to the numbering system mentioned above (bottom left is 0 and starts going across to the right)

This function just gets the postion of the `object` (ie. the checkerboard) corners in the objects frame and then sorts them according to the pointids list/vector.  This is to aling the board points with the detected image points so that the algorithm can match them

In [ ]:
def get_board_object_points(board, pointids):
    corners = board.chessboardCorners
    return corners[pointids.reshape((len(pointids)))]


In [ ]:
images

# The real work 

In [ ]:
# set up list variables to store the results
allIds = []
allCorners = []
allObjectPoints = []
allImagePoints = []

# get files by looping over the images array
for ind, image in enumerate(images):

    filename = image
    img = cv.imread(filename)
    
    print('{}:{}'.format(ind, image))
    
    # find the corners of the checkerboard between the aruco markers
    (corners, 
     ids, 
     imsize) = find_checkerboard_corners(img, board, clipLimit=0.0, verbose=False)

    # once we have the common ids get the board object points that correspond to the common ids
    object_pts = get_board_object_points(board, ids )

    # store the results for this image
    allCorners.append(corners)
    allIds.append(ids)
    allObjectPoints.append(object_pts)
    
    fig, axs = plt.subplots(1,1,figsize=(36,12))
    axs.imshow(cv.aruco.drawDetectedCornersCharuco(img, corners, ids))
    plt.show()


# VIP Note:
The analysis uses different components in the openCV library.  Some components use image size as (width, height) while others use (height,width).  Here we create two variables:
- imsize (height, width)
- imsize_t (width, height)

In [ ]:
imsize

In [ ]:
# this is the transposed imsize used in the rectify and remap
imsize_t = (imsize[1], imsize[0])
imsize_t

# Camera calibration

The stereo claibration works much better if the cameras are calibrated.  We should have done this first, but do it now

In [ ]:
def calibrate_camera(allCorners,allIds, board, imsize):
    """
    Calibrates the camera using the dected corners.
    """
    print("CAMERA CALIBRATION")

    cameraMatrixInit = np.array([[ 1000.,    0., imsize[0]/2.],
                                 [    0., 1000., imsize[1]/2.],
                                 [    0.,    0.,           1.]])

    distCoeffsInit = np.zeros((5,1))
    #flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
    flags = (cv.CALIB_RATIONAL_MODEL)
    (ret, camera_matrix, distortion_coefficients0,
     rotation_vectors, translation_vectors,
     stdDeviationsIntrinsics, stdDeviationsExtrinsics,
     perViewErrors) = cv.aruco.calibrateCameraCharucoExtended(
                      charucoCorners=allCorners,
                      charucoIds=allIds,
                      board=board,
                      imageSize=imsize,
                      cameraMatrix=cameraMatrixInit,
                      distCoeffs=distCoeffsInit,
                      flags=flags,
                      criteria=(cv.TERM_CRITERIA_EPS & cv.TERM_CRITERIA_COUNT, 100000, 1e-9))

    return ret, camera_matrix, distortion_coefficients0, rotation_vectors, translation_vectors

# Check that this works for charuco boards

In [ ]:
retL, mtxL, distL, rvecsL, tvecsL = calibrate_camera(allCorners, allIds, board, imsize)

In [ ]:
retR, mtxR, distR, rvecsR, tvecsR = calibrate_camera(allCornersR, allIdsR, board, imsize)

In [ ]:
mtxL

In [ ]:
distL

In [ ]:
distR
# this does not look right why is distR is far diffferent?

In [ ]:
# save the results in lists that can be indexed by imgIndexL
ret = [retL, retR]
mtx = [mtxL, mtxR]
dist = [distL, distR]
rvecs = [rvecsL, rvecsR]
tvecs = [tvecsL, tvecsR]

In [ ]:
import pickle

camera_calibrationL = {'camera_name':'Left Camera','ret':retL, 'mtx':mtxL, 'dist':distL , 'rvecs':rvecsL, 'tvecs':tvecsL}
pickle.dump(camera_calibrationL, open('LeftCameraCalibration.p', 'wb'))

camera_calibrationR = {'camera_name':'Right Camera','ret':retR, 'mtx':mtxR, 'dist':distR , 'rvecs':rvecsR, 'tvecs':tvecsR}
pickle.dump(camera_calibrationR, open('RightCameraCalibration.p', 'wb'))

camera_calibration_combined = {'camera_name':'Combined','ret':ret, 'mtx':mtx, 'dist':dist , 'rvecs':rvecs, 'tvecs':tvecs}
pickle.dump(camera_calibration_combined, open('CombinedCameraCalibration.p', 'wb'))




In [ ]:
i=3 # select image id
side = imgIndexL

plt.figure(figsize=(18,18))
frame = cv2.imread(images[i][side])
img_undist = cv2.undistort(frame,mtx[side],dist[side],None)
plt.subplot(1,2,1)
plt.imshow(frame)
plt.title("Raw image")
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(img_undist)
plt.title("Corrected image")
plt.axis("off")
plt.show()

In [ ]:
ret

# Now for the big moment

From [here](https://vgg.fiit.stuba.sk/2015-02/2783/)


In [ ]:
stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100000, 1e-11)

In [ ]:
stereocalib_flags = (cv2.CALIB_FIX_ASPECT_RATIO 
        | cv2.CALIB_ZERO_TANGENT_DIST 
        | cv2.CALIB_SAME_FOCAL_LENGTH 
        | cv2.CALIB_RATIONAL_MODEL 
        | cv2.CALIB_FIX_K3 
        | cv2.CALIB_FIX_K4 
        | cv2.CALIB_FIX_K5)

In [ ]:
(retval, cameraMatrixL,
 distCoeffsL, cameraMatrixR, 
 distCoeffsR, R, T, E, F) = cv2.stereoCalibrate(allObjectPoints, allCornersL, allCornersR, 
                                                mtx[imgIndexL], dist[imgIndexL], 
                                                mtx[imgIndexR], dist[imgIndexR], imsize,
                                                criteria = stereocalib_criteria,
                                                flags = stereocalib_flags)

In [ ]:
retval

In [ ]:
T

In [ ]:
R

In [ ]:
cameraMatrixR

In [ ]:
mtx[imgIndexL]

In [ ]:
rectify_flags = cv.CALIB_ZERO_DISPARITY

In [ ]:

(R_L, R_R, 
 P_L, P_R, 
 Q, 
 validPixROIL, validPixROIR) = cv2.stereoRectify(mtx[imgIndexL], dist[imgIndexL], 
                                                 mtx[imgIndexR], dist[imgIndexR], 
                                                 imsize_t, R, T, flags=rectify_flags, alpha=0)

In [ ]:
validPixROIR

Documentation for initUndistortRectifyMap [here](https://docs.rs/opencv/0.22.1/opencv/calib3d/fn.stereo_rectify_camera.html)

In [ ]:
mapxL, mapyL = cv2.initUndistortRectifyMap(mtxL, distL, R_L, P_L, imsize_t, cv.CV_32FC1)

In [ ]:
mapxR, mapyR = cv2.initUndistortRectifyMap(mtxR, distR, R_R, P_R, imsize_t, cv.CV_32FC1)

In [ ]:
recitfication_maps = {'mapxL':mapxL, 'mapyL':mapyL, 'mapxR':mapxR, 'mapyR':mapyR }
pickle.dump(recitfication_maps, open('Rectification_maps.p', 'wb'))

# Now we can check

In [ ]:
image_number=2 # select image id

imgL = cv2.imread(images[image_number][0])
imgR = cv2.imread(images[image_number][1])

In [ ]:
# remap the images to rectify them
imgL_rect = cv2.remap(imgL, mapxL, mapyL, interpolation=cv.INTER_LINEAR  )
imgR_rect = cv2.remap(imgR, mapxR, mapyR, interpolation=cv.INTER_LINEAR  )

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(18,6))
axs[0].imshow(imgL_rect)
axs[1].imshow(imgR_rect)
plt.show()

In [ ]:
combined = np.concatenate((imgL_rect, imgR_rect), axis=1)

In [ ]:
fig, axs = plt.subplots(figsize=(18,6))
axs.imshow(combined)
plt.show()

In [ ]:
def draw_horizontal_lines(img, n, color=(255,255,255), thickness=2):
    increment = int(img.shape[0]/(n+1))
    img_ = img.copy()
    
    for i in range(1,n+1):
        cv2.line(img_, (0, i*increment), (img.shape[1], i*increment), color, thickness, 1)
    
    return img_

In [ ]:
combined.shape

In [ ]:
c = draw_horizontal_lines(combined, 30, color=(255,255,255), thickness=5 )

In [ ]:
fig, axs = plt.subplots(figsize=(20,10))
axs.imshow(c)
plt.show()

In [ ]:
def find_charuco_marker_corners(img, board, clipLimit=3.0, verbose=False):

    # These are parameters used by the cv2.cornerSubPix function
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)


    # increase the contraxt
    img = increase_contrast(img, clipLimit=clipLimit)
 
    # convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)

    if verbose:
        print('Found {} aruco marker corners'.format(len(ids)))
        
    
    #  now find the central points of the aruco markers by averaging the four corner points
    if  len(ids) > 0:
        # SUB PIXEL DETECTION
        for corner in corners:
            if verbose:
                print('Sub pixel optimization:')
                print(corner)
            cv2.cornerSubPix(gray, corner,
                             winSize = (15,15),
                             zeroZone = (-1,-1),
                             criteria = criteria)
            if verbose:
                print(corner)
                print('+++')




    return np.array(corners), ids, gray.shape

        

In [ ]:
charucoCornersL, charucoIdsL, imsize =  find_charuco_marker_corners(imgL_rect, 
                                                    board, clipLimit=3.0, 
                                                    verbose=False, 
                                                    )
charucoCornersR, charucoIdsR, imsize =  find_charuco_marker_corners(imgR_rect, 
                                                    board, clipLimit=3.0, 
                                                    verbose=False, 
                                                    )

In [ ]:
plot_corners = np.array(charucoCornersL).reshape((-1,1,2))
plot_ids = np.arange(plot_corners.shape[0]).reshape(-1,1)

fig, axs = plt.subplots(figsize=(36,12))
axs.imshow(cv2.aruco.drawDetectedCornersCharuco(imgL_rect, plot_corners, plot_ids))
plt.show()

In [ ]:
# find the common 
(cornersL_com, 
  idsL_com, 
  cornersR_com, 
      idsR_com) = find_common_corners(charucoCornersL, charucoIdsL, 
                                                 charucoCornersR, charucoIdsR, 
                                                 verbose=True)




In [ ]:
plot_corners = np.array(cornersL_com).reshape((-1,1,2))
plot_ids = np.arange(plot_corners.shape[0]).reshape(-1,1)

fig, axs = plt.subplots(figsize=(36,12))
axs.imshow(cv2.aruco.drawDetectedCornersCharuco(imgL_rect, plot_corners, plot_ids))
plt.show()

In [ ]:
def find_aruco_center(corners, ids):
    # find the center point of the 4 corner of the aruco markers
    centers = []
    for cnrs in corners:
        center = np.array((np.average(cnrs[0][:,0]), np.average(cnrs[0][:,1])))
        centers.append(center)
    centers = np.array(centers)
    centers = np.array(centers).reshape((-1,1,2))
    center_ids = np.arange(centers.shape[0]).reshape(-1,1)
    
    return centers, ids

In [ ]:
centersL, center_idsL = find_aruco_center(cornersL_com, idsL_com)
centersR, center_idsR = find_aruco_center(cornersR_com, idsR_com)

In [ ]:
fig, axs = plt.subplots(figsize=(36,12))
axs.imshow(cv2.aruco.drawDetectedCornersCharuco(imgL_rect, centersL, center_idsL))
plt.show()

In [ ]:
# now we have the common center in the two pictures 

# calculate the dispersion between them

# calculate the distance 


In [ ]:
disparity = (centersL-centersR)[:,0][:,0]

In [ ]:
disparity

## Camera features:
- sensor size = 3.68 x 2.76 mm  
- sensor resolution  = 3280 × 2464
- focal length = 3.04 mm

$$ d_{mm} = \frac{pix \times 3.68}{3280} $$

In [ ]:
# convert pixels to mm
disparity_mm = disparity*3.68/3280


In [ ]:
disparity_mm

The depth can now be found
$$ Z = \frac{T \times f}{d_{mm}} $$

In [ ]:
f = 3.04
T = 40

In [ ]:
depth = f*T/disparity_mm

In [ ]:
depth

In [ ]:
x = centersL[:,0,0]
y = centersL[:,0,1]

annot_font = {'fontname':'Arial', 'size':'14','weight':'bold'}

fig, axs = plt.subplots(figsize=(36,12))
axs.imshow(imgL_rect)
axs.scatter(x,y, color='r')
for i, txt in enumerate(depth):
    axs.annotate('{}'.format(int(txt)), (x[i]+20, y[i]-10), color='r', **annot_font)
for i, txt in enumerate(center_idsL.reshape(-1,)):
    axs.annotate('{}'.format(int(txt)), (x[i]+20, y[i]+30), color='g',  **annot_font)


plt.show()

In [ ]:
# for each point in each image add the distance to a df

In [ ]:
import pandas as pd

In [ ]:
stereo_distance = pd.DataFrame(columns=['ArucoId', 'DisparityPix', 'DisparityMM', 'Distance', 'File'])

In [ ]:
for i, ids in enumerate(center_idsL):
    print(i, ids)
    ser = pd.Series()
    ser['ArucoId']  = int(ids[0])
    ser['DisparityPix'] = disparity[i]
    ser['DisparityMM'] = disparity_mm[i]
    ser['Distance'] = depth[i]
    ser['File'] = images[image_number][0]

    stereo_distance = stereo_distance.append(ser, ignore_index=True)

In [ ]:
stereo_distance